In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from models import ConvMixer, MlpMixer
from torchvision.datasets import CIFAR10, ImageFolder
from torch.utils.data import DataLoader 
from torchvision import transforms as T

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.empty_cache())
print(torch.cuda.memory_summary(0))

In [ ]:
batch_size=2
hdim=1536
depth=24

epochs=1

scale=0.75
reprob=0.25
ra_m=8
ra_n=1
jitter=0.1
psize=2
conv_ks=5
wd=0.01
clip_norm=True
lr_max=0.01
workers=2

In [ ]:
from pathlib import Path
import os

if not Path('data/tiny-imagenet-200').exists():
    os.system('wget http://cs231n.stanford.edu/tiny-imagenet-200.zip -P data')
    os.system('unzip -qq data/tiny-imagenet-200.zip -d data')

DATA_DIR = 'data/tiny-imagenet-200' # Original images come in shapes of [3,64,64]

# Define training and validation data paths
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'val')

traindata = ImageFolder(TRAIN_DIR, transform=T.Compose([
    T.RandomResizedCrop(64, scale=(scale, 1.0)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
]))

valdata = ImageFolder(VALID_DIR, transform=T.Compose([
    T.Resize(64),
    T.CenterCrop(64),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
]))

trainloader = DataLoader(traindata, batch_size=batch_size, shuffle=True, num_workers=workers)
valloader = DataLoader(valdata, batch_size=batch_size, shuffle=False, num_workers=workers)

print(len(traindata))
print(len(valdata))

In [ ]:
# cifar10_mean = (0.4914, 0.4822, 0.4465)
# cifar10_std = (0.2471, 0.2435, 0.2616)
# train_transform = T.Compose([
#     T.RandomResizedCrop(32, scale=(scale, 1.0), ratio=(1.0, 1.0)),
#     T.RandomHorizontalFlip(p=0.5),
#     T.RandAugment(num_ops=ra_n, magnitude=ra_m),
#     T.ColorJitter(jitter, jitter, jitter),
#     T.ToTensor(),
#     T.Normalize(cifar10_mean, cifar10_std),
#     T.RandomErasing(p=reprob)
# ])

# test_transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize(cifar10_mean, cifar10_std)
# ])
# traindata = CIFAR10(root="data", train=True, download=True, transform=train_transform)
# testdata = CIFAR10(root="data", train=False, download=True, transform=test_transform)
# trainloader = DataLoader(traindata, batch_size=batch_size, shuffle=True, num_workers=workers)
# testloader = DataLoader(testdata, batch_size=batch_size, shuffle=False, num_workers=workers)

In [ ]:
# model = ConvMixer(hdim, depth, kernel_size=9, patch_size=7, n_classes=1000)
model = MlpMixer(num_blocks=depth, embed_dim=hdim)
model = model.cuda()
lr_schedule = lambda t: np.interp([t], [0, epochs*2//5, epochs*4//5, epochs], 
                                  [0, lr_max, lr_max/20.0, 0])[0]

opt = optim.AdamW(model.parameters(), lr=lr_max, weight_decay=wd)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

In [ ]:
print(f"batch_size: {batch_size}, hdim: {hdim}, depth: {depth}")
print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")
print(f"Total params size in gb: {sum(p.element_size()*p.nelement() for p in model.parameters())/1024**3:.4f}GB")
preload_mem = 0.
load_mem = 0.
forward_mem = 0.
transfered = 0
step_time = []
record_mem = 3
record_time = list(range(4, 4+100))
for epoch in range(epochs):
    start = time.time()
    train_loss, train_acc, n = 0, 0, 0
    for i, (X, y) in enumerate(trainloader):
        if i in record_time: start_step = time.time()
        if i == record_mem: preload_mem = torch.cuda.memory_allocated(0)/1024**3
        model.train()
        X, y = X.cuda(), y.cuda()
        if i == record_mem:
            load_mem = torch.cuda.memory_allocated(0)/1024**3
            transfered += X.element_size() * X.nelement()
            transfered += y.element_size() * y.nelement()

        lr = lr_schedule(epoch + (i + 1)/len(trainloader))
        opt.param_groups[0].update(lr=lr)

        opt.zero_grad()
        # with torch.cuda.amp.autocast():
        output = model(X)
        loss = criterion(output, y)
        if i == record_mem:
            forward_mem = torch.cuda.memory_allocated(0)/1024**3
            transfered += loss.element_size() * loss.nelement()

        loss.backward()
        if clip_norm:
            # scaler.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # scaler.step(opt)
        # scaler.update()
        opt.step()

        train_loss += loss.item() * y.size(0)
        train_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)
        # print(f"step {i} of {len(trainloader)}")
        if i in record_time: step_time.append(time.time() - start_step)
        if i == record_time[-1]: print(f'avg step time: {np.mean(step_time):.4f} +- {np.std(step_time)}s')

        if i == record_mem:
            print(f'preload_mem: {preload_mem:.4f}GB, load_mem: {load_mem:.4f}GB, forward_mem: {forward_mem:.4f}GB, transfered: {transfered/1024**2:.4f}MB')
        
    model.eval()
    test_acc, m = 0, 0
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X, y = X.cuda(), y.cuda()
            with torch.cuda.amp.autocast():
                output = model(X)
            test_acc += (output.max(1)[1] == y).sum().item()
            m += y.size(0)

    print(f'[ConvMixer] Epoch: {epoch} | Train Acc: {train_acc/n:.4f}, Test Acc: {test_acc/m:.4f}, Time: {time.time() - start:.1f}, lr: {lr:.6f}')
